<a href="https://colab.research.google.com/github/Akiane-cloudy/Introduction-to-Data-science-competition/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to data science

- Competition: `Titanic`
- Link: https://www.kaggle.com/competitions/titanic